In [1]:
import requests
import json
import pandas as pd
import numpy as np
import datetime
import wbdata
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook" # use "pio.renderers" to see the default renderer

## Objective
- Build table with yearly debt for a selection of countries. 
- Metric is called "Total External debt stocks" here: https://datatopics.worldbank.org/debt/ids/countryanalytical/ARG
- Data source: https://worldbank.github.io/debt-data/api-guide/ids-api-guide-python-1.html
- Source name: "International Debt Statistics", id:6
- Indicator name: "External debt stocks, total (DOD, current US$)", code: "DT.DOD.DECT.CD"
- Country selection: "Low & middle income" (LMY)
- World Bank Data API docs: https://wbdata.readthedocs.io/en/latest/wbdata_library.html

### Check data coordinates

In [6]:
# Make sure source exists
wbdata.get_source(source_id=6)

  id  name
----  -----------------------------
   6  International Debt Statistics

In [21]:
# Make sure indicator exists in source
ind = wbdata.get_indicator(source=6)
for i in ind:
    if i["id"] == "DT.DOD.DECT.CD":
        print("Found: "+ (str(i)))

Found: {'id': 'DT.DOD.DECT.CD', 'name': 'External debt stocks, total (DOD, current US$)', 'unit': '', 'source': {'id': '6', 'value': 'International Debt Statistics'}, 'sourceNote': 'Total external debt is debt owed to nonresidents repayable in currency, goods, or services. Total external debt is the sum of public, publicly guaranteed, and private nonguaranteed long-term debt, use of IMF credit, and short-term debt. Short-term debt includes all debt having an original maturity of one year or less and interest in arrears on long-term debt. Data are in current U.S. dollars.', 'sourceOrganization': 'World Bank, International Debt Statistics.', 'topics': [{'id': '3', 'value': 'Economy & Growth'}, {'id': '20', 'value': 'External Debt'}]}


In [22]:
# Print all selected countries
wbdata.get_country(incomelevel='LMY')

id    name
----  ------------------------------
AFG   Afghanistan
AGO   Angola
ALB   Albania
ARG   Argentina
ARM   Armenia
ASM   American Samoa
AZE   Azerbaijan
BDI   Burundi
BEN   Benin
BFA   Burkina Faso
BGD   Bangladesh
BGR   Bulgaria
BIH   Bosnia and Herzegovina
BLR   Belarus
BLZ   Belize
BOL   Bolivia
BRA   Brazil
BTN   Bhutan
BWA   Botswana
CAF   Central African Republic
CHN   China
CIV   Cote d'Ivoire
CMR   Cameroon
COD   Congo, Dem. Rep.
COG   Congo, Rep.
COL   Colombia
COM   Comoros
CPV   Cabo Verde
CRI   Costa Rica
CUB   Cuba
DJI   Djibouti
DMA   Dominica
DOM   Dominican Republic
DZA   Algeria
ECU   Ecuador
EGY   Egypt, Arab Rep.
ERI   Eritrea
ETH   Ethiopia
FJI   Fiji
FSM   Micronesia, Fed. Sts.
GAB   Gabon
GEO   Georgia
GHA   Ghana
GIN   Guinea
GMB   Gambia, The
GNB   Guinea-Bissau
GNQ   Equatorial Guinea
GRD   Grenada
GTM   Guatemala
GUY   Guyana
HND   Honduras
HTI   Haiti
IDN   Indonesia
IND   India
IRN   Iran, Islamic Rep.
IRQ   Iraq
JAM   Jamaica
JOR   Jordan
KAZ   Kaza

## Create dataframe
### Obs
- Here I encountered the following error when specifying a source in the arguments, (source=6):
"RuntimeError: Got error 175 (Invalid format): The indicator was not found. It may have been deleted or archived."
- Time range is not required but I thought it maybe better not to leave rely on defaults

In [14]:
countries = [i['id'] for i in wbdata.get_country(incomelevel='LMY')]                                                                                            
indicators = {"DT.DOD.DECT.CD":"TotalExternalDebtUSD"}
timeSelection = (datetime.datetime(2009, 1, 1), datetime.datetime(2018, 12, 31))
df = wbdata.get_dataframe(indicators, data_date=timeSelection,country=countries, convert_date=True)   
df

TotalExternalDebtUSD
country     date                            
Afghanistan 2018-01-01          2.678760e+09
            2017-01-01          2.751987e+09
            2016-01-01          2.596050e+09
            2015-01-01          2.596917e+09
            2014-01-01          2.529865e+09
...                                      ...
Zimbabwe    2013-01-01          8.415959e+09
            2012-01-01          8.886963e+09
            2011-01-01          7.392833e+09
            2010-01-01          6.762813e+09
            2009-01-01          6.156585e+09

[1370 rows x 1 columns]